In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(11672, 128, 128, 3)

In [8]:
y.shape

(11672, 6)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(9337, 128, 128, 3)

In [11]:
X_test.shape

(2335, 128, 128, 3)

In [12]:
y_train.shape

(9337, 6)

In [13]:
y_test.shape

(2335, 6)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

6

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 12:37:36.433581: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


292/292 [==============================] - 132s 446ms/step - loss: 0.6453 - accuracy: 0.7646 - val_loss: 1.8168 - val_accuracy: 0.5739
Epoch 2/10
292/292 [==============================] - 135s 463ms/step - loss: 0.2572 - accuracy: 0.9085 - val_loss: 1.0600 - val_accuracy: 0.7203
Epoch 3/10
292/292 [==============================] - 128s 440ms/step - loss: 0.1721 - accuracy: 0.9400 - val_loss: 0.3307 - val_accuracy: 0.8869
Epoch 4/10
292/292 [==============================] - 133s 456ms/step - loss: 0.1385 - accuracy: 0.9526 - val_loss: 0.1494 - val_accuracy: 0.9520
Epoch 5/10
292/292 [==============================] - 113s 385ms/step - loss: 0.1060 - accuracy: 0.9637 - val_loss: 0.2594 - val_accuracy: 0.9148
Epoch 6/10
292/292 [==============================] - 79s 271ms/step - loss: 0.0965 - accuracy: 0.9674 - val_loss: 0.1951 - val_accuracy: 0.9370
Epoch 7/10
292/292 [==============================] - 80s 275ms/step - loss: 0.0825 - accuracy: 0.9725 - val_loss: 0.2151 - val_accuracy

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 96.49%


In [19]:
cnn_scores

[0.12405312806367874, 0.9648822546005249]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

73/73 [==============================] - 4s 54ms/step
Accuracy: 96.49%
Precision: 96.72%
Recall: 96.49%
F1 score: 96.51%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-apel+pepper.csv', index=False)

+------+----------------------------------------------------------------------------+-------------------------------+-------------------------------+
|      | Filename                                                                   | True Labels                   | Predicted Labels              |
|------+----------------------------------------------------------------------------+-------------------------------+-------------------------------|
|    0 | 4e6311ab-8927-4147-9857-a669f2268cb1___JR_FrgE.S 8726.JPG                  | Apple___Black_rot             | Apple___Apple_scab            |
|    1 | c5a5ffe1-6687-412b-9a06-31c9aee8066b___JR_FrgE.S 3037.JPG                  | Apple___Black_rot             | Apple___Black_rot             |
|    2 | a48cbfdf-653b-4ab0-8f63-2d6977ec1014___RS_HL 5639.JPG                      | Apple___healthy               | Apple___healthy               |
|    3 | 8ae37f10-377d-44a3-ad94-f271a95673c8___RS_HL 7297.JPG                      | Apple___health

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-apel+pepper.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------------+-------------------------------+-------------------------------+
|      | Filename                                                                   | True Labels                   | Predicted Labels              |
|------+----------------------------------------------------------------------------+-------------------------------+-------------------------------|
|    0 | 4e6311ab-8927-4147-9857-a669f2268cb1___JR_FrgE.S 8726.JPG                  | Apple___Black_rot             | Apple___Apple_scab            |
|   47 | 89699048-94c3-4389-9a86-13e73dfd23c6___RS_HL 7257.JPG                      | Apple___healthy               | Apple___Apple_scab            |
|   73 | 5b1f2adb-5065-4b7f-8b51-bc5695ad5ebb___FREC_C.Rust 9802_new30degFlipLR.JPG | Apple___Cedar_apple_rust      | Apple___Apple_scab            |
|   80 | a2024303-1e78-4f1b-80b4-3f02f9f5dceb___RS_HL 7707_flipTB.JPG            

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

292/292 [==============================] - 17s 56ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 117.54034376144409 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

73/73 [==============================] - 3s 44ms/step
Random Forest Classifier Accuracy: 98.50%
Random Forest Classifier Precision: 0.99
Random Forest Classifier Recall: 0.99
Random Forest Classifier F1 Score: 0.99


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-apel+pepper.csv', index=False)

+------+----------------------------------------------------------------------------+-------------------------------+-------------------------------+
|      | Filename                                                                   | True Labels                   | Predicted Labels              |
|------+----------------------------------------------------------------------------+-------------------------------+-------------------------------|
|    0 | 4e6311ab-8927-4147-9857-a669f2268cb1___JR_FrgE.S 8726.JPG                  | Apple___Black_rot             | Apple___Black_rot             |
|    1 | c5a5ffe1-6687-412b-9a06-31c9aee8066b___JR_FrgE.S 3037.JPG                  | Apple___Black_rot             | Apple___Black_rot             |
|    2 | a48cbfdf-653b-4ab0-8f63-2d6977ec1014___RS_HL 5639.JPG                      | Apple___healthy               | Apple___healthy               |
|    3 | 8ae37f10-377d-44a3-ad94-f271a95673c8___RS_HL 7297.JPG                      | Apple___health

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-apel+pepper.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------------+-------------------------------+-------------------------------+
|      | Filename                                                                   | True Labels                   | Predicted Labels              |
|------+----------------------------------------------------------------------------+-------------------------------+-------------------------------|
|   47 | 89699048-94c3-4389-9a86-13e73dfd23c6___RS_HL 7257.JPG                      | Apple___healthy               | Apple___Apple_scab            |
|  174 | dc9bcd28-1f28-430c-9f05-9895174a2805___JR_HL 5975.JPG                      | Pepper,_bell___healthy        | Pepper,_bell___Bacterial_spot |
|  281 | 3235339f-7b7b-423a-850a-0fd58ded4725___JR_FrgE.S 2791.JPG                  | Apple___Black_rot             | Apple___Cedar_apple_rust      |
|  336 | e7f0bd90-56b8-4d09-a406-9c4786eaca3a___RS_HL 7738_flipTB.JPG            